In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [87]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

# Обработаем дата сет:

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [89]:
data =  pd.read_csv('./data_set/weather_prediction_dataset.csv')
data.T

,0,1,2,3,4,5,6,7,8,9,...,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653
DATE,2.000010e+07,2.000010e+07,2.000010e+07,2.000010e+07,2.000010e+07,2.000011e+07,2.000011e+07,2.000011e+07,2.000011e+07,2.000011e+07,...,2.009122e+07,2.009122e+07,2.009122e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.010010e+07
MONTH,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.000000e+00
BASEL_cloud_cover,8.000000e+00,8.000000e+00,5.000000e+00,7.000000e+00,5.000000e+00,3.000000e+00,8.000000e+00,4.000000e+00,8.000000e+00,8.000000e+00,...,8.000000e+00,8.000000e+00,6.000000e+00,4.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,8.000000e+00,7.000000e+00,8.000000e+00
BASEL_humidity,8.900000e-01,8.700000e-01,8.100000e-01,7.900000e-01,9.000000e-01,8.500000e-01,8.400000e-01,7.900000e-01,8.800000e-01,9.100000e-01,...,9.300000e-01,9.300000e-01,7.100000e-01,8.100000e-01,8.000000e-01,8.200000e-01,9.200000e-01,9.200000e-01,9.300000e-01,9.300000e-01
BASEL_pressure,1.028600e+00,1.031800e+00,1.031400e+00,1.026200e+00,1.024600e+00,1.024400e+00,1.026700e+00,1.024800e+00,1.024300e+00,1.033700e+00,...,9.984000e-01,9.924000e-01,1.002500e+00,1.012000e+00,1.011800e+00,1.008400e+00,1.002800e+00,9.979000e-01,9.958000e-01,9.965000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TOURS_global_radiation,2.500000e-01,1.700000e-01,2.700000e-01,1.100000e-01,3.900000e-01,5.500000e-01,2.700000e-01,1.800000e-01,4.300000e-01,5.800000e-01,...,2.200000e-01,1.500000e-01,6.700000e-01,5.000000e-01,5.800000e-01,2.200000e-01,2.400000e-01,2.400000e-01,5.800000e-01,1.100000e-01
TOURS_precipitation,4.000000e-02,1.600000e-01,0.000000e+00,4.400000e-01,4.000000e-02,2.000000e-01,0.000000e+00,1.400000e-01,2.000000e-02,4.000000e-02,...,0.000000e+00,3.600000e-01,0.000000e+00,0.000000e+00,8.000000e-02,1.500000e+00,4.000000e-01,1.000000e+00,2.000000e-02,0.000000e+00
TOURS_temp_mean,8.500000e+00,7.900000e+00,8.100000e+00,8.600000e+00,8.000000e+00,7.100000e+00,6.000000e+00,8.200000e+00,5.600000e+00,1.800000e+00,...,3.800000e+00,4.800000e+00,5.400000e+00,2.600000e+00,4.600000e+00,6.200000e+00,1.040000e+01,1.000000e+01,8.500000e+00,5.000000e-01
TOURS_temp_min,7.200000e+00,6.600000e+00,6.600000e+00,6.400000e+00,6.400000e+00,3.500000e+00,3.300000e+00,5.300000e+00,3.700000e+00,-1.500000e+00,...,2.000000e+00,1.800000e+00,2.500000e+00,-1.100000e+00,-5.000000e-01,1.800000e+00,6.200000e+00,8.700000e+00,6.200000e+00,-7.000000e-01


## Выберем 1 город

In [90]:
data = data.iloc[:, :11]
data.T

,0,1,2,3,4,5,6,7,8,9,...,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653
DATE,2.000010e+07,2.000010e+07,2.000010e+07,2.000010e+07,2.000010e+07,2.000011e+07,2.000011e+07,2.000011e+07,2.000011e+07,2.000011e+07,...,2.009122e+07,2.009122e+07,2.009122e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.009123e+07,2.010010e+07
MONTH,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.200000e+01,1.000000e+00
BASEL_cloud_cover,8.000000e+00,8.000000e+00,5.000000e+00,7.000000e+00,5.000000e+00,3.000000e+00,8.000000e+00,4.000000e+00,8.000000e+00,8.000000e+00,...,8.000000e+00,8.000000e+00,6.000000e+00,4.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,8.000000e+00,7.000000e+00,8.000000e+00
BASEL_humidity,8.900000e-01,8.700000e-01,8.100000e-01,7.900000e-01,9.000000e-01,8.500000e-01,8.400000e-01,7.900000e-01,8.800000e-01,9.100000e-01,...,9.300000e-01,9.300000e-01,7.100000e-01,8.100000e-01,8.000000e-01,8.200000e-01,9.200000e-01,9.200000e-01,9.300000e-01,9.300000e-01
BASEL_pressure,1.028600e+00,1.031800e+00,1.031400e+00,1.026200e+00,1.024600e+00,1.024400e+00,1.026700e+00,1.024800e+00,1.024300e+00,1.033700e+00,...,9.984000e-01,9.924000e-01,1.002500e+00,1.012000e+00,1.011800e+00,1.008400e+00,1.002800e+00,9.979000e-01,9.958000e-01,9.965000e-01
BASEL_global_radiation,2.000000e-01,2.500000e-01,5.000000e-01,6.300000e-01,5.100000e-01,5.600000e-01,2.000000e-01,5.400000e-01,1.100000e-01,6.000000e-02,...,3.600000e-01,1.300000e-01,1.600000e-01,5.400000e-01,3.700000e-01,2.800000e-01,2.200000e-01,7.000000e-02,1.700000e-01,8.000000e-02
BASEL_precipitation,3.000000e-02,0.000000e+00,0.000000e+00,3.500000e-01,7.000000e-02,0.000000e+00,0.000000e+00,0.000000e+00,6.500000e-01,9.000000e-02,...,6.100000e-01,8.900000e-01,6.000000e-02,0.000000e+00,1.800000e-01,4.200000e-01,1.680000e+00,1.540000e+00,5.700000e-01,5.600000e-01
BASEL_sunshine,0.000000e+00,0.000000e+00,3.700000e+00,6.900000e+00,3.700000e+00,5.700000e+00,0.000000e+00,4.300000e+00,0.000000e+00,0.000000e+00,...,1.100000e+00,0.000000e+00,2.000000e-01,5.900000e+00,2.300000e+00,3.000000e-01,2.000000e-01,0.000000e+00,1.000000e-01,0.000000e+00
BASEL_temp_mean,2.900000e+00,3.600000e+00,2.200000e+00,3.900000e+00,6.000000e+00,4.200000e+00,4.700000e+00,5.600000e+00,4.600000e+00,2.400000e+00,...,2.900000e+00,3.900000e+00,6.200000e+00,4.000000e-01,1.000000e+00,3.200000e+00,4.500000e+00,8.500000e+00,6.600000e+00,2.900000e+00
BASEL_temp_min,1.600000e+00,2.700000e+00,1.000000e-01,5.000000e-01,3.800000e+00,1.900000e+00,1.800000e+00,4.100000e+00,3.800000e+00,1.400000e+00,...,6.000000e-01,2.300000e+00,1.200000e+00,-2.000000e+00,-3.900000e+00,1.000000e+00,2.400000e+00,7.500000e+00,4.300000e+00,-2.000000e-01


In [91]:
data = data.drop('DATE', axis=1)

In [92]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,3644,3645,3646,3647,3648,3649,3650,3651,3652,3653
MONTH,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,...,12.0000,12.0000,12.0000,12.000,12.0000,12.0000,12.0000,12.0000,12.0000,1.0000
BASEL_cloud_cover,8.0000,8.0000,5.0000,7.0000,5.0000,3.0000,8.0000,4.0000,8.0000,8.0000,...,8.0000,8.0000,6.0000,4.000,7.0000,7.0000,7.0000,8.0000,7.0000,8.0000
BASEL_humidity,0.8900,0.8700,0.8100,0.7900,0.9000,0.8500,0.8400,0.7900,0.8800,0.9100,...,0.9300,0.9300,0.7100,0.810,0.8000,0.8200,0.9200,0.9200,0.9300,0.9300
BASEL_pressure,1.0286,1.0318,1.0314,1.0262,1.0246,1.0244,1.0267,1.0248,1.0243,1.0337,...,0.9984,0.9924,1.0025,1.012,1.0118,1.0084,1.0028,0.9979,0.9958,0.9965
BASEL_global_radiation,0.2000,0.2500,0.5000,0.6300,0.5100,0.5600,0.2000,0.5400,0.1100,0.0600,...,0.3600,0.1300,0.1600,0.540,0.3700,0.2800,0.2200,0.0700,0.1700,0.0800
BASEL_precipitation,0.0300,0.0000,0.0000,0.3500,0.0700,0.0000,0.0000,0.0000,0.6500,0.0900,...,0.6100,0.8900,0.0600,0.000,0.1800,0.4200,1.6800,1.5400,0.5700,0.5600
BASEL_sunshine,0.0000,0.0000,3.7000,6.9000,3.7000,5.7000,0.0000,4.3000,0.0000,0.0000,...,1.1000,0.0000,0.2000,5.900,2.3000,0.3000,0.2000,0.0000,0.1000,0.0000
BASEL_temp_mean,2.9000,3.6000,2.2000,3.9000,6.0000,4.2000,4.7000,5.6000,4.6000,2.4000,...,2.9000,3.9000,6.2000,0.400,1.0000,3.2000,4.5000,8.5000,6.6000,2.9000
BASEL_temp_min,1.6000,2.7000,0.1000,0.5000,3.8000,1.9000,1.8000,4.1000,3.8000,1.4000,...,0.6000,2.3000,1.2000,-2.000,-3.9000,1.0000,2.4000,7.5000,4.3000,-0.2000
BASEL_temp_max,3.9000,4.8000,4.8000,7.5000,8.6000,6.9000,6.2000,8.4000,5.7000,3.8000,...,4.4000,5.7000,10.6000,4.000,5.5000,4.8000,10.0000,11.5000,7.9000,4.8000


In [93]:
data.to_numpy()

array([[ 1.  ,  8.  ,  0.89, ...,  2.9 ,  1.6 ,  3.9 ],
       [ 1.  ,  8.  ,  0.87, ...,  3.6 ,  2.7 ,  4.8 ],
       [ 1.  ,  5.  ,  0.81, ...,  2.2 ,  0.1 ,  4.8 ],
       ...,
       [12.  ,  8.  ,  0.92, ...,  8.5 ,  7.5 , 11.5 ],
       [12.  ,  7.  ,  0.93, ...,  6.6 ,  4.3 ,  7.9 ],
       [ 1.  ,  8.  ,  0.93, ...,  2.9 , -0.2 ,  4.8 ]])

# Размер данных для обучения:

In [94]:
size = 5

# Напишем класс dataset

In [95]:
class MyDataset(torch.utils.data.Dataset):
    """Some Information about MyDataset"""
    def __init__(self, data, data_output):
        super(MyDataset, self).__init__()
        self.data = data 
        self.data_out = data_output

    def __getitem__(self, index):
        data = self.data[index:index+self.data_out]
        data = data.reshape((1, 10*self.data_out))[0]
        lable = self.data[index+self.data_out]
        return torch.Tensor(data), torch.Tensor(lable)

    def __len__(self):
        return len(self.data)-self.data_out

In [96]:
dataset = MyDataset(data.to_numpy(), size)

In [97]:
batch_size = 10
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, shuffle=True)

# Опишем модель:

In [131]:
class MyModule(nn.Module):
    """Some Information about MyModule"""
    def __init__(self, data_inp):
        super(MyModule, self).__init__()
        self.pipe = nn.Sequential(
            nn.Linear(10*data_inp, 200),
            nn.ReLU(),

            nn.Linear(200, 500),
            nn.ReLU(),

            nn.Linear(500, 500),
            nn.ReLU(),

            nn.Dropout(),

            nn.Linear(500, 10)
        ) 



    def forward(self, x):
        x = self.pipe(x)
        return x

In [132]:
model = MyModule(size)

In [133]:
model.to(device)

MyModule(
  (pipe): Sequential(
    (0): Linear(in_features=50, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=500, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.5, inplace=False)
    (7): Linear(in_features=500, out_features=10, bias=True)
  )
)

# Обучение

In [134]:
from tqdm import tqdm

In [135]:
def train(epoch, model, optimizer, criterion, dataloader, save = False):
    model.train()
    # loop over the dataset multiple times
    for epoch in range(epoch):
        running_loss = 0.0
        for datainp in (pdbar := tqdm(dataloader)):
            inputs, labels = datainp
            inputs = inputs.to(device)
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)


            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pdbar.set_description(f'epoch: {epoch}\tloss: {running_loss:.3F}')
        if save:
            torch.save(model, f'my_model_epoch_{epoch}.pt')
    print('Finished Training')
    model.eval()

In [136]:
epoch = 50
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()
training = True

In [137]:
if (training):
    train(epoch, model, optimizer, criterion, dataloader)

epoch: 49	loss: 1923.438: 100%|██████████| 365/365 [00:03<00:00, 119.20it/s]

Finished Training


# Средняя ошибка

In [140]:
err = model(dataset[0][0].to(device))-dataset[0][1].to(device)
for ind in range(1, len(dataset)):
    err += (model(dataset[ind][0].to(device))-dataset[ind][1].to(device))
err/len(dataset)

tensor([ 0.0198, -0.1571,  0.0116,  0.0775,  0.4242, -0.2221,  0.5156,  0.9521,
         0.8459,  1.1267], device='cuda:0', grad_fn=<DivBackward0>)

# Средняя квадратичная ошибка

In [144]:
err = model(dataset[0][0].to(device))-dataset[0][1].to(device)
for ind in range(1, len(dataset)):
    err += (model(dataset[ind][0].to(device))-dataset[ind][1].to(device))**2
torch.sqrt(err/len(dataset))

tensor([2.0751, 2.2623, 0.1183, 0.1253, 0.8130, 0.6092, 3.8894, 2.3376, 2.4724,
        3.2493], device='cuda:0', grad_fn=<SqrtBackward0>)

# Итог результаты топ 